#### New QAT frontdoor getting started (EXPERIMENTAL)

This is a demo of the new QAT 'frontdoor' which uses the experimental QAT pipelines.

We aren't using this in production yet and your mileage may vary.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# This is required to support both the script and notebook versions of this file
from pathlib import Path

if nb_dir := globals().get("_dh"):
    notebooks_dir = Path(nb_dir[0]).parents[0].resolve()
else:
    notebooks_dir = Path(__file__).parents[1].resolve()

Import QAT and instantiate a qat instance

In [ ]:
from qat import QAT

qat = QAT()

Default pipelines are setup by default...

In [ ]:
qat.pipelines

Define a program...

In [ ]:
src = """
OPENQASM 3;
bit[2] c;
qubit[2] q;
h q;
measure q -> c;
"""

Compile it with the default pipeline...

In [ ]:
pkg, metrics = qat.compile(src)

Execute the compiled program with the default pipeline...

In [ ]:
res, metrics = qat.execute(pkg)
print(res)

Comile and execute a program with a different pipeline...

In [ ]:
pkg, metrics = qat.compile(src, pipeline="echo16")
res, metrics = qat.execute(pkg, pipeline="echo16")

QAT pipelines can also be configured with YAML...

In [ ]:
qat = QAT(qatconfig=str(notebooks_dir) + "/qatconfig.yaml")
qat.pipelines

The yaml looks like this... (It's not very flexible yet it will be)

In [ ]:
import pathlib

import yaml

print(yaml.dump(yaml.safe_load(pathlib.Path(notebooks_dir, "qatconfig.yaml").read_text())))

Change the default pipeline

In [ ]:
qat.pipelines.set_default("echo8-alt")

# Default shot number is now highet than pipeline max limit
try:
    inst, metrics = qat.compile(src)
except ValueError as ex:
    print(ex)

Create a custom config and set a lower shot count

In [ ]:
from compiler_config.config import CompilerConfig

config = CompilerConfig(repeats=8)
inst, metrics = qat.compile(src, compiler_config=config)
res, metrics = qat.execute(inst, compiler_config=config)
print(res)

Run a program as a one liner...

In [ ]:
res, metrics = qat.run(src, compiler_config=config)
print(res)

Make a custom pipeline

In [ ]:
from qat import Pipeline

In [ ]:
from qat.backend.waveform_v1 import WaveformV1Backend
from qat.engines.waveform_v1 import EchoEngine
from qat.frontend import AutoFrontend
from qat.middleend import DefaultMiddleend
from qat.purr.backends.echo import get_default_echo_hardware
from qat.runtime.simple import SimpleRuntime

model = get_default_echo_hardware(qubit_count=16)
P = Pipeline(
    name="mycoolnewpipeline",
    frontend=AutoFrontend(model),
    middleend=DefaultMiddleend(model),
    backend=WaveformV1Backend(model),
    runtime=SimpleRuntime(engine=EchoEngine()),
    model=model,
)

Compile and execute against the new pipeline

In [ ]:
pkg, metrics = qat.compile(src, compiler_config=config, pipeline=P)
res, metrics = qat.execute(pkg, compiler_config=config, pipeline=P)
print(res)

Keep it around for later...

In [ ]:
qat.pipelines.add(P)
P.name

Now it's available by name

In [ ]:
pkg, metrics = qat.compile(src, compiler_config=config, pipeline="mycoolnewpipeline")
res, metrics = qat.execute(pkg, compiler_config=config, pipeline="mycoolnewpipeline")
print(res)